In [ ]:
import s4sim.hardware.config
c=s4sim.hardware.config.get_example()

In [ ]:
import s4sim

In [ ]:
s4sim.__version__

In [ ]:
c.data["telescopes"]["SAT0"]["fwhm"]

In [ ]:
c.data["telescopes"]["SAT3"]["fwhm"].keys()

In [ ]:
c.data["telescopes"]["SAT3"].keys()

In [ ]:
c.data["bands"].keys()

In [ ]:
c.data["bands"]["LFL1"]

In [ ]:
import h5py

In [ ]:
h5py.get_config().track_order = True

In [ ]:
s4 = h5py.File("../cmbs4_tophat.h5", "w")

In [ ]:
import numpy as np

# Sensitivity from Decadal Survey Report

For verification purposes we want to compare simulation results to the expected performance as detailed in the DSR.
The DSR is available in tex format in the private Github repository: https://github.com/CMB-S4/DecadalSurveyReport

Numbers extracted on 26 January 2021 from version `83d362e2301b80444ed09c29293998cf5917d899`

[LAT Pole polarization](https://github.com/CMB-S4/DecadalSurveyReport/blob/83d362e2301b80444ed09c29293998cf5917d899/ScienceMeasurement/sciencemeasurement.tex#L336-L344)

In [ ]:
"""Frequency & 20 & 30 & 40 & 85 & 95 & 145 & 155 & 220 & 270
$Q/U$ rms ($\mu$K-arcmin)  & 8.4 & 3.5 & 4.5 & 0.88 & 0.78 & 1.2 & 1.3 & 3.5 & 6.0"""

# NET and number of detectors per tube

From the spreadsheet at https://docs.google.com/spreadsheets/d/1X0x8wYhjHdI_WPjhgbtyVlVQrX86tTgxgqEUhan2Hxg/edit?pli=1#gid=1404929524

Extracted on July 2nd 2020

In [ ]:
NUM_DET = {}
SAT_NUM_DET_STRING = "296	296	3556	3556	3556	3556	8594	8594"
for sat_band, num_det in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [ ]:
NET = {}
SAT_NET_STRING = "177	224	270	309	238	331	747	1281"
for band, net in zip(c.data["telescopes"]["SAT3"]["fwhm"].keys(), SAT_NET_STRING.split()):
    NET[band] = float(net)

In [ ]:
LAT_NUM_DET_STRING = "216	384	384	3456	3456	3456	3456"
for sat_band, num_det in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NUM_DET_STRING.split()):
    NUM_DET[sat_band] = int(num_det)

In [ ]:
c.data["telescopes"]["LAT0"]["fwhm"].keys()

In [ ]:
LAT_NET_STRING = "473.0	429.0	261.0	305.0	385.0	792.0	1990.0"
for band, net in zip(c.data["telescopes"]["LAT0"]["fwhm"].keys(), LAT_NET_STRING.split()):
    NET[band] = float(net)

In [ ]:

pole_NET = {}
pole_LAT_NET_STRING = "473.0 445.0 320.0   347.0   374.0   664.0   1609.0"
for band, net in zip(list(c.data["telescopes"]["LAT0"]["fwhm"].keys()), pole_LAT_NET_STRING.split()):
    pole_NET[band] = float(net)

In [ ]:
NET

In [ ]:
pole_NET

In [ ]:
for i, (band, v) in enumerate(c.data["bands"].items()):
    print("## " + band)
    print("Bandpass [GHz] low,center,high: ", v["low"], v["center"], v["high"])
    try:
        tel = "SAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    except:
        tel = "LAT"
        fwhm = c.data["telescopes"][tel + "0"]["fwhm"][band]
    print("Beam [arcmin]: ", fwhm)
    s4.create_group(band)
    s4[band].attrs["band"] = band
    s4[band].attrs["telescope"] = tel
    s4[band].attrs["center_frequency_GHz"] = v["center"]
    s4[band].attrs["fwhm_arcmin"] = fwhm
    s4[band].attrs["detectors_per_tube"] = NUM_DET[band]
    s4[band].attrs["NET_uKsqrts"] = NET[band]
    if tel == "LAT":
        s4[band].attrs["pole_NET_uKsqrts"] = pole_NET[band]
    s4[band].attrs["index"] = i
    s4[band].create_dataset('bandpass_frequency_GHz', data=np.linspace(v["low"], v["high"], 10))
    s4[band].create_dataset("bandpass_weight", data=np.ones(10, dtype=np.float))
    print("Num det per tube", s4[band].attrs["detectors_per_tube"])

In [ ]:
s4.keys()

In [ ]:
s4["MFL1"].get("bandpass_frequency_GHz")

In [ ]:
s4.close()

In [ ]:
pwd